# RGB to XYZ 
Transformation from RGB to XYZ is a simple matrix multiplication<br>
$[X Y Z] = M \times [R G B]$<br>
The Matrix M is a standard matrix based on the standard whitepoint of monitors. HOWEVER, when we measure the _specific_ whitepoint of our monitor, we can refine the conversion.<br>

In [50]:
# import libraries

# use the os
import os
import glob
# math and data structure
import numpy as np
import pandas as pd
# plotting
import matplotlib.pyplot as plt
# image processing
import cv2

In [42]:
stim_dir = os.path.join('..','stimuli')
# taken from original - added alpha channel
true_color_stim_dir = os.path.join(stim_dir,'true_color')
if not os.path.exists(true_color_stim_dir):
    os.mkdir(true_color_stim_dir)
inverted_lab_stim_dir = os.path.join(stim_dir,'inverted_lab')
if not os.path.exists(inverted_lab_stim_dir):
    os.mkdir(inverted_lab_stim_dir)
inverted_lab_stim_dir_refined = os.path.join(stim_dir,'inverted_lab_refined')
if not os.path.exists(inverted_lab_stim_dir_refined):
    os.mkdir(inverted_lab_stim_dir_refined)
    
foreground_background_mask_dir = os.path.join(stim_dir,'foreground_background_masks')
if not os.path.exists(foreground_background_mask_dir):
    os.mkdir(foreground_background_mask_dir)
color_mask_dir = os.path.join(stim_dir, 'color_masks')
if not os.path.exists(color_mask_dir):
    os.mkdir(color_mask_dir)

In [28]:
# M taken from Measurement at CIN 2nd floor
def rgb2xyz(img):
    # sanity checks that the provided image is actually a X*Y*3 image
    if type(img) is not np.ndarray:
        print('Input image was not a numpy array')
        return False
    if len(img.shape)!=3:
        print('Array is not three dimensional')
        return False
    if img.shape[2]!=3:
        print('The images color channel does not have three (R-G-B) dimensions')
        return False
    # sanity check that image is one of the opencv supported file types
    if img.dtype=='float32':
        cap = 1
    elif img.dtype=='uint8':
        cap = 255
    elif img.dtype=='float64':
        cap=1
    else:
        print('Unsupported data type')
        raise AssertionError()
    
    # RGB to XYZ transformation matrix derived from monitor calibration
    M = np.array([[0.4040,0.3602,0.1987],
                  [0.2041,0.7307,0.0652],
                  [0.0050,0.0731,1.0797]])
    
    # tensordot does the matrix multiplication along the right dimension
    xyz_img = np.tensordot(img,M.T, axes=1)
    # xyz_img = np.clip(xyz_img,0,cap)
    xyz_img = xyz_img.astype(img.dtype)
    return(xyz_img)

In [2]:
# function f and the xyz to lab transformation are taken from here:
# https://docs.opencv.org/3.4/de/d25/imgproc_color_conversions.html
def f(t):
    output = t.copy()
    output[t>0.008856] = t[t>0.008856]**(1/3)
    output[t<=0.008856] = t[t<=0.008856]*7.787+(16/116)
    return output
def xyz2lab(xyz):
    if xyz.dtype=='float32':
        delta = 0
    elif xyz.dtype=='uint8':
        delta = 128
        xyz = xyz.astype(np.float32)/255
    else:
        print('Unsupported data type')
        print(xyz.dtype)
        delta = 0
        
    # X_n, Y_n, Z_n taken from measurement at CIN 2nd floor
    X_n = 149.0/100
    Y_n = 154.8/100
    Z_n = 179.3/100
    
    X,Y,Z = cv2.split(xyz)
    X = X/X_n
    Y = Y/Y_n
    Z = Z/Z_n
    L = np.zeros_like(Y)
    a = np.zeros_like(X)
    b = np.zeros_like(Z)
    
    L[Y>0.008856] = Y[Y>0.008856]**(1/3)*116-16
    L[Y<=0.008856] = 903.3*Y[Y<=0.008856]
    
    a = 500*(f(X) - f(Y)) + delta
    b = 200*(f(Y) - f(Z)) + delta
    
    lab = cv2.merge((L,a,b))
    return lab

In [3]:
def rgb2lab(rgb):
    xyz = rgb2xyz(rgb)
    lab = xyz2lab(xyz)
    return lab

In [4]:
# backtransformation was taken frome here:
# https://github.com/scikit-image/scikit-image/blob/main/skimage/color/colorconv.py
def lab2xyz(lab):
    L,a,b = cv2.split(lab)
    
    Y = L.copy()
    X = a.copy()
    Z = b.copy()
    
    Y = (L+16.0) / 116.0
    X = (a / 500) + Y
    Z = Y - (b / 200.0)
    
    Z[Z < 0] = 0
    
    xyz = cv2.merge((X,Y,Z))
    mask = xyz > 0.2068966
    xyz[mask] = xyz[mask]**3.0
    xyz[~mask] = (xyz[~mask] - 16.0 / 116.) / 7.787
    
    xyz[:,:,0] *= 149.0/100
    xyz[:,:,1] *= 154.8/100
    xyz[:,:,2] *= 179.3/100
    
    return xyz

In [25]:
def xyz2rgb(xyz):
    if xyz.dtype == np.float32:
        cap = 1.0
    elif xyz.dtype == np.uint8:
        cap = 255
    else:
        print('Unsupported data type')
        raise AssertionError()
        
    M = np.array([[0.4040,0.3602,0.1987],
                  [0.2041,0.7307,0.0652],
                  [0.0050,0.0731,1.0797]])
    
    M_inv = np.linalg.inv(M)
    
    rgb = np.tensordot(xyz,M_inv.T, axes=1)
    rgb = np.clip(rgb,0,cap)
    rgb = rgb.astype(xyz.dtype)
    return(rgb)

In [6]:
def lab2rgb(lab):
    xyz = lab2xyz(lab)
    rgb = xyz2rgb(xyz)
    return rgb

In [16]:
# get a list of all stimuli in the directory
stimuli_full_path = glob.glob(os.path.join(true_color_stim_dir,'*.png'))
stimuli_full_path.sort()

# extract the image name from the stimulus path
stimuli = [os.path.basename(stim) for stim in stimuli_full_path]

In [41]:
for stim in stimuli:
    # get true color stimlus directory
    true_stim_dir = os.path.join(true_color_stim_dir,stim)
    # get inverted color stimulus directory
    inv_stim_dir = os.path.join(inverted_lab_stim_dir,stim)
    # get stimulus mask
    mask_dir = os.path.join(foreground_background_mask_dir,stim)
    
    # read in the image
    true_img = cv2.imread(true_stim_dir)
    # convert image from BGR to RGB
    true_rgb = cv2.cvtColor(true_img, cv2.COLOR_BGR2RGB)
    # convert image from unsigned integer to float and change range to 0-1
    true_rgb = true_rgb.astype(np.float32) / 255.0
    
    # read in the mask
    mask_img = cv2.imread(mask_dir)
    # the mask has three channels and values are 0 or 255 -> make it a 2D boolean mask
    mask = mask_img[:,:,0].astype(bool)
    
    # convert image from RGB to LAB
    true_lab = rgb2lab(true_rgb)
    
    # invert the image
    L,a,b = cv2.split(true_lab)
    a_inv = -a
    b_inv = -b
    
    inv_lab = cv2.merge((L,a_inv,b_inv))
    
    # convert image from inverted lab back to rgb
    inv_rgb = lab2rgb(inv_lab)
    # convert image back into unsigned int
    inv_rgb = (inv_rgb*255.0).astype(np.uint8)
    
    # change to BGR and add alpha channel to image 
    inv_rgba = cv2.cvtColor(inv_rgb, cv2.COLOR_RGB2BGRA)
    
    # make background invisible
    inv_rgba[~mask] = np.array([255,255,255,0],dtype='uint8')
    
    # save image
    cv2.imwrite(inv_stim_dir, inv_rgba)

### Refined inverted images
In the previous steps we inverted the color of the whole object
Now we create inverted images with the refined masks

# TODO!

# Representative colors of stimuli
For each stimulus get a representative pixel of the true color and invered color stimulus

In [51]:
# dictionary to store results in
rep_pixel_dict = {
    'stimuli': [],
    'true_R': [],
    'true_G': [],
    'true_B': [],
    'inv_R': [],
    'inv_G': [],
    'inv_B': [],
}

stimulus_selection = [
    'blue_nivea.png',
    'blue_pool.png',
    'blue_sign.png',
    'green_brokkoli_1.png',
    'green_frog_1.png',
    'green_lettuce_1.png',
    'orange_basketball.png',
    'orange_carrots.png',
    'orange_pumpkin.png',
    'red_fire_extinguisher_1.png',
    'red_strawberry.png',
    'red_tomato.png',
    'yellow_banana.png',
    'yellow_chicken.png',
    'yellow_corn.png'
]

for stim in stimulus_selection:
    rep_pixel_dict['stimuli'].append(stim)
    # get paths of stimuli
    true_path = os.path.join(true_color_stim_dir,stim)
    inv_path = os.path.join(inverted_lab_stim_dir,stim)
    # read in the original and inverted stimuli
    true_color_img = cv2.imread(true_path)
    inv_color_img = cv2.imread(inv_path)
    # convert the image from BRG (opencv default) to RGB (everybodies default)
    true_color_img = cv2.cvtColor(true_color_img, cv2.COLOR_BGR2RGB)
    inv_color_img = cv2.cvtColor(inv_color_img, cv2.COLOR_BGR2RGB)
    
    # read in the mask of the stimulus
    mask_path = os.path.join(color_mask_dir, stim)
    mask_img = cv2.imread(mask_path)
    # the mask is saved as an image with three channels with either 0 or 255 values -> convert to boolean mask
    mask_img = mask_img[:,:,0].astype(bool)
    
    # extract the pixels within the mask
    true_pixels = true_color_img[mask_img==1]
    inv_pixels = inv_color_img[mask_img==1]
    
    # get the representative pixel by searching for the pixel values appearing most often
    # true pixel
    # unique_colors, counts = np.unique(true_pixels,axis=0,return_counts=True)
    # most_frequent_indices = np.flip(np.argsort(counts))
    # # we need to convert the pixel from np.uint8 to int. Otherwise the transversion to the dataframe messes things up
    # most_frequent_true_color = unique_colors[most_frequent_indices[0]].astype(int)
    most_frequent_true_color = true_pixels.mean(axis=0)
    rep_pixel_dict['true_R'].append(most_frequent_true_color[0])
    rep_pixel_dict['true_G'].append(most_frequent_true_color[1])
    rep_pixel_dict['true_B'].append(most_frequent_true_color[2])
    
    # inverted pixel
    # unique_colors, counts = np.unique(inv_pixels,axis=0,return_counts=True)
    # most_frequent_indices = np.flip(np.argsort(counts))
    # # we need to convert the pixel from np.uint8 to int. Otherwise the transversion to the dataframe messes things up
    # most_frequent_inverted_color = unique_colors[most_frequent_indices[0]].astype(int)
    most_frequent_inverted_color = inv_pixels.mean(axis=0)
    rep_pixel_dict['inv_R'].append(most_frequent_inverted_color[0])
    rep_pixel_dict['inv_G'].append(most_frequent_inverted_color[1])
    rep_pixel_dict['inv_B'].append(most_frequent_inverted_color[2])
    
# convert the dictionary to a dataframe
pixel_df = pd.DataFrame(data=rep_pixel_dict, columns=rep_pixel_dict.keys())
pixel_df.to_csv(os.path.join(stim_dir,'representative_pixels.csv'))

### Luminance sanity check
Changing the color in LAB space resulted in pixels not representable in RGB space.<br>
In turn the conversion back results in different colors with different luminances<br>
<br>
Check the actual differences of luminance

In [24]:
lum_diff_dir = os.path.join(results_dir, 'lum_diff')
if not os.path.exists(lum_diff_dir):
    os.mkdir(lum_diff_dir)
    
lum_diff_sum = []
lum_diff_percent = []

for stim in stimuli:
    true_path = os.path.join(true_color_stim_dir, stim)
    inv_path = os.path.join(inverted_lab_stim_dir, stim)
    mask_path = os.path.join(mask_dir, stim)
    mask_img = cv2.imread(mask_path)
    mask_img = mask_img[:,:,0].astype(bool)

    true_color_img = cv2.imread(true_path)
    true_lab = cv2.cvtColor(true_color_img, cv2.COLOR_BGR2LAB)
    true_L, a, b = cv2.split(true_lab)
    
    inv_color_img = cv2.imread(inv_path)
    inv_lab = cv2.cvtColor(inv_color_img, cv2.COLOR_BGR2LAB)
    inv_L, a, b = cv2.split(inv_lab)

    true_L_f = true_L.astype(np.float32)
    inv_L_f = inv_L.astype(np.float32)
    
    lum_diff = (true_L_f-inv_L_f)[mask_img==1]
    lum_diff_sum.append(lum_diff.sum())
    lum_diff_percent.append(sum(lum_diff!=0)/len(lum_diff))
    
    fig = plt.figure()
    plt.hist(lum_diff, bins=40)
    plt.savefig(os.path.join(lum_diff_dir,stim))
    plt.close(fig=fig)